In [11]:
import lstm_functions as lf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import data_prep as dp

import torch
import model as m
from itertools import product


In [ ]:
da_price_data = pd.read_csv("../models_santi/data_santi/data_da_23.csv")

# Cross-validation parameters
n_splits = 5
gammas = [0.1, 0.5, 1.0]
model_params = {
    "input_size": da_price_data.shape[1],
    "hidden_size": 16,
    "num_layers": 3,
    "dropout": 0.2,
    "past_horizon": 10,
    "forecast_horizon": 5,
    "batch_size": 16
}

class MockLSTMForecaster:
    def __init__(self, *args, **kwargs):
        pass

    def forecast(self, length):
        # Generate random forecasts
        return np.random.uniform(50, 150, length)


def cross_validation_test():
    result = lf.cross_validation(
        model_type="da",
        n_splits=n_splits,
        price_data=da_price_data,
        gammas=gammas,
        model_params=model_params
    )
    print(result)

# Replace LSTM class with the mock for testing
cross_validation = lf.cross_validation.__globals__.copy()
cross_validation['m.LSTM_multivariate_input_multi_step_forecaster'] = MockLSTMForecaster

cross_validation_test()


AttributeError: 'LSTM_multivariate_input_multi_step_forecaster' object has no attribute 'shape'

In [3]:
class MockLSTMForecaster:
    def __init__(self, *args, **kwargs):
        pass

    def forecast(self, length):
        # Generate random forecasts
        return np.random.uniform(50, 150, length)


In [2]:
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, data, past_horizon, forecast_horizon, target_col):
        """
        Args:
            data (pd.DataFrame): DataFrame containing the time series data.
            past_horizon (int): Number of past timesteps to use as input.
            forecast_horizon (int): Number of future timesteps to predict.
            target_col (str): Column name for the target variable.
        """
        self.data = data
        self.past_horizon = past_horizon
        self.forecast_horizon = forecast_horizon
        self.target_col = target_col

    def __len__(self):
        return len(self.data) - self.past_horizon - self.forecast_horizon

    def __getitem__(self, idx):
        # Input sequence
        x = self.data.iloc[idx:idx + self.past_horizon].values
        # Target sequence
        y = self.data.iloc[idx + self.past_horizon:idx + self.past_horizon + self.forecast_horizon][self.target_col].values
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_model(model, train_loader, valid_loader, epochs, lr, device):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs.squeeze(-1), y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        valid_loss = 0.0
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = model(x_batch)
                loss = criterion(outputs.squeeze(-1), y_batch)
                valid_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss / len(train_loader):.4f}, Validation Loss: {valid_loss / len(valid_loader):.4f}")

    return model


In [4]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error

def cross_validation_lstm(data, target_col, past_horizon, forecast_horizon, n_splits, model_params, training_params, device):
    """
    Args:
        data (pd.DataFrame): DataFrame with the time series data.
        target_col (str): Name of the target variable column.
        past_horizon (int): Number of past timesteps for input.
        forecast_horizon (int): Number of future timesteps to predict.
        n_splits (int): Number of splits for TimeSeriesSplit.
        model_params (dict): Parameters for the LSTM model.
        training_params (dict): Parameters for training (epochs, learning rate, batch size).
        device (str): 'cuda' or 'cpu'.
    Returns:
        pd.DataFrame: Cross-validation results.
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = []

    for fold, (train_index, valid_index) in enumerate(tscv.split(data)):
        train_data = data.iloc[train_index]
        valid_data = data.iloc[valid_index]

        # Create datasets and dataloaders
        train_dataset = TimeSeriesDataset(train_data, past_horizon, forecast_horizon, target_col)
        valid_dataset = TimeSeriesDataset(valid_data, past_horizon, forecast_horizon, target_col)
        train_loader = DataLoader(train_dataset, batch_size=training_params['batch_size'], shuffle=False)
        valid_loader = DataLoader(valid_dataset, batch_size=training_params['batch_size'], shuffle=False)

        # Initialize the model
        model = m.LSTM_multivariate_input_multi_step_forecaster(
            input_size=model_params['input_size'],
            hidden_size=model_params['hidden_size'],
            num_layers=model_params['num_layers'],
            dropout=model_params['dropout'],
            past_horizon=past_horizon,
            forecast_horizon=forecast_horizon
        )

        # Train the model
        model = train_model(model, train_loader, valid_loader, training_params['epochs'], training_params['lr'], device)

        # Evaluate on validation set
        model.eval()
        predictions, targets = [], []
        with torch.no_grad():
            for x_batch, y_batch in valid_loader:
                x_batch = x_batch.to(device)
                y_pred = model(x_batch).squeeze(-1).cpu().numpy()
                predictions.append(y_pred)
                targets.append(y_batch.numpy())
        
        predictions = np.concatenate(predictions)
        targets = np.concatenate(targets)

        mape = mean_absolute_percentage_error(targets, predictions)
        results.append({"fold": fold, "MAPE": mape})
        print(f"Fold {fold}, MAPE: {mape:.4f}")

    return pd.DataFrame(results)


In [19]:
def grid_search_lstm(data, target_col, past_horizon, forecast_horizon, n_splits, param_grid, training_params, device):
    from itertools import product
    results = []

    # Generate parameter combinations
    param_combinations = list(product(*param_grid.values()))
    param_names = list(param_grid.keys())

    for params in param_combinations:
        # Create parameter dictionary for this combination
        param_dict = dict(zip(param_names, params))

        # Update model_params and training_params
        model_params = {
            "input_size": 13,  # Assuming all features are input
            "hidden_size": param_dict["hidden_size"],
            "num_layers": param_dict["num_layers"],
            "dropout": param_dict["dropout"]
        }
        training_params["lr"] = param_dict["lr"]

        # Run cross-validation
        cv_results = cross_validation_lstm(
            data=data,
            target_col=target_col,
            past_horizon=past_horizon,
            forecast_horizon=forecast_horizon,
            n_splits=n_splits,
            model_params=model_params,
            training_params=training_params,
            device=device
        )

        # Compute average MAPE across folds
        avg_mape = cv_results["MAPE"].mean()

        # Store the results with parameters
        results.append({"params": param_dict, "avg_mape": avg_mape})
        print(f"Tested params: {param_dict}, Avg MAPE: {avg_mape:.4f}")

    # Return all results as a DataFrame
    return pd.DataFrame(results)


In [20]:
import pandas as pd
import numpy as np

# data preprocessing
scaler = MinMaxScaler()

data = dp.create_dataframe()

# input_data = data.drop(columns=["DA"]).values  # Extract features
# target_data = data["DA"].values.reshape(-1, 1)  # Extract target

# scaled_input = scaler.fit_transform(input_data)
# scaled_target = scaler.fit_transform(target_data)


# Model and training parameters
model_params = {
    "input_size": 12,  # Number of features
    "hidden_size": 100,
    "num_layers": 3,
    "dropout": 0.2
}

training_params = {
    "epochs": 3,
    "lr": 0.001,
    "batch_size": 32
}

param_grid = {
    "hidden_size": [32, 64, 128],
    "num_layers": [1, 2, 3],
    "dropout": [0.1, 0.2],
    "lr": [0.001, 0.0005]
}

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

grid_search_results = grid_search_lstm(
    data=data,
    target_col="Price1",
    past_horizon=24,
    forecast_horizon=24,
    n_splits=10,
    param_grid=param_grid,
    training_params=training_params,
    device=device
)

print(grid_search_results)


# Run cross-validation
cv_results = cross_validation_lstm(
    data=data,
    target_col="Price1",
    past_horizon=24,
    forecast_horizon=24,
    n_splits=10,
    model_params=model_params,
    training_params=training_params,
    device=device
)

print(cv_results)


Found 365 marginalpdbc files
Found 365 precious files
Processed 8760 rows from marginalpdbc files
Processed 8760 rows from precious files


/opt/anaconda3/envs/deep_learning_ex_2/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/3, Train Loss: 8862.1548, Validation Loss: 9564.8330
Epoch 2/3, Train Loss: 8498.3294, Validation Loss: 9132.6260
Epoch 3/3, Train Loss: 8086.8916, Validation Loss: 8648.1135
Fold 0, MAPE: 0.9050


/opt/anaconda3/envs/deep_learning_ex_2/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/3, Train Loss: 9124.2887, Validation Loss: 7874.5540
Epoch 2/3, Train Loss: 8184.4201, Validation Loss: 6891.2329
Epoch 3/3, Train Loss: 7074.7526, Validation Loss: 5797.7572
Fold 1, MAPE: 2367429191663616.0000


...

/opt/anaconda3/envs/deep_learning_ex_2/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/3, Train Loss: 8218.2157, Validation Loss: 7401.3272
Epoch 2/3, Train Loss: 5756.0743, Validation Loss: 4708.5633
Epoch 3/3, Train Loss: 3542.1333, Validation Loss: 2709.6223
Fold 8, MAPE: 2315973335973888.0000


/opt/anaconda3/envs/deep_learning_ex_2/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/3, Train Loss: 8091.6374, Validation Loss: 6908.7610
Epoch 2/3, Train Loss: 5263.3981, Validation Loss: 4036.2252
Epoch 3/3, Train Loss: 3002.1794, Validation Loss: 2270.0836
Fold 9, MAPE: 752100514463744.0000
Tested params: {'hidden_size': 128, 'num_layers': 1, 'dropout': 0.2, 'lr': 0.0005}, Avg MAPE: 702756373397504.0000
Epoch 1/3, Train Loss: 8841.2256, Validation Loss: 9530.9399
Epoch 2/3, Train Loss: 8466.1300, Validation Loss: 9090.3741
Epoch 3/3, Train Loss: 8044.3406, Validation Loss: 8594.6929
Fold 0, MAPE: 0.9027
Epoch 1/3, Train Loss: 9099.1333, Validation Loss: 7846.0763
Epoch 2/3, Train Loss: 8156.3370, Validation Loss: 6869.1056
Epoch 3/3, Train Loss: 7056.4304, Validation Loss: 5796.1448
Fold 1, MAPE: 2365392034988032.0000
Epoch 1/3, Train Loss: 8526.3799, Validation Loss: 7851.1828
Epoch 2/3, Train Loss: 7052.0531, Validation Loss: 6182.4034
Epoch 3/3, Train Loss: 5398.9636, Validation Loss: 4506.9676
Fold 2, MAPE: 197979084423168.0000
Epoch 1/3, Train Loss: 822

KeyboardInterrupt: 